In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import requests
import pandas as pd

In [2]:
#función para guardar las imagenes
folders=['elpais', 'elconfidencial', 'eldiario', 'elmundo']
miruta='/Users/ESTEBAN/Pictures/'
def load_requests(source_url, folder):
    r = requests.get(source_url, stream = True)
    if r.status_code == 200:
        aSplit = source_url.split('/')
        if folder == 'elconfidencial':
            aSplit[-1] = aSplit[-1].split('?')[0]
        ruta = miruta + folder + '/' + aSplit[-1]
        output = open(ruta,'wb')
        for chunk in r:
            output.write(chunk)
        output.close()


In [3]:
#extraemos los articulos de las portadas digitales de los medios seleccionados
now = datetime.datetime.now()
medios = ["https://elpais.com/", 
          "https://www.elconfidencial.com", 
          "https://www.eldiario.es/", 
          "https://www.elmundo.es/"]
articulos=[]
for medio in medios:
    articulos.append(BeautifulSoup(requests.get(medio).content, "lxml")('article'))


In [4]:
#extraemos los nombres de los medios
nombres=[]
for medio in medios:
    nombres.append(BeautifulSoup(requests.get(medio).content, "lxml").title.string)

In [5]:
medio, time, titulo_articulo, autor_articulo, href_articulo, titulo_imagen, imagen =[],[],[],[],[],[],[]

#extraemos los datos de los articulos de la portada de "El Pais"
for articulo in articulos[0]:
    medio.append(nombres[0])
    time.append(now.isoformat())
    titulo_articulo.append(articulo.h2.a.string)
    if articulo.find('span', itemprop="name"):
        autor_articulo.append(articulo.find('span', itemprop="name").a.string)
    else:
        autor_articulo.append('')
    if articulo.h2.a['href'][1]=='/':
        href_articulo.append('https:'+articulo.h2.a['href'])
    elif articulo.h2.a['href'][0]=='/' and  articulo.h2.a['href'][1]!='/':
        href_articulo.append('https://elpais.com'+articulo.h2.a['href'])
    else:
        href_articulo.append(articulo.h2.a['href'])
    if articulo.img and articulo.img['alt']:
        titulo_imagen.append(articulo.img['alt'])
        imagen.append(articulo.img['data-src'])
        source_url= "https:" + articulo.img['data-src']
        folder = folders[0]
        load_requests(source_url, folder)
    else:
        titulo_imagen.append('')
        imagen.append('') 
        
#extraemos los datos de los articulos de la portada de "El Confidencial"
for articulo in articulos[1]:
    if articulo.a:
        medio.append(nombres[1])
        time.append(now.isoformat())
        titulo_articulo.append(articulo.a['title'])
        if articulo.strong:
            autor_articulo.append(articulo.strong.string)
        else:
            autor_articulo.append('')
        href_articulo.append(articulo.a['href'])
        if articulo.img and 'data-original' in articulo.img.attrs:
            titulo_imagen.append(articulo.img['title'])
            imagen.append(articulo.img['data-original'])
            source_url= articulo.img['data-original']
            folder = folders[1]
            load_requests(source_url, folder)
        elif articulo.img and 'src' in articulo.img.attrs:
            titulo_imagen.append(articulo.img['title'])
            imagen.append(articulo.img['src'])
            source_url= articulo.img['src']
            folder = folders[1]
            load_requests(source_url, folder)
        else:
            titulo_imagen.append('')
            imagen.append('')
           
#extraemos los datos de los articulos de la portada de "eldiario.es"
for articulo in articulos[2]:
    if articulo.a and 'title' in articulo.a.attrs:
        medio.append(nombres[2])
        time.append(now.isoformat())
        titulo_articulo.append(articulo.a['title'])
        if articulo.address:
            if articulo.address.a:
                autor_articulo.append(articulo.address.a.string)
            else:
                autor_articulo.append(articulo.address.string)
        else:
            autor_articulo.append('')
        if articulo.a['href'][0]=='/':
            href_articulo.append('https://www.eldiario.es'+articulo.a['href'])
        else:
            href_articulo.append(articulo.a['href'])
        if articulo.img and 'data-original' in articulo.img.attrs:
            titulo_imagen.append(articulo.img['alt'])
            imagen.append('https://www.eldiario.es'+articulo.img['data-original'])
            source_url= 'https://www.eldiario.es'+articulo.img['data-original']
            folder = folders[2]
            load_requests(source_url, folder)
        elif articulo.img and 'src' in articulo.img.attrs and 'data-original' not in articulo.img.attrs:
            titulo_imagen.append(articulo.img['alt'])
            imagen.append('https://www.eldiario.es'+articulo.img['src'])
            source_url= 'https://www.eldiario.es'+articulo.img['src']
            folder = folders[2]
            load_requests(source_url, folder)
        else:
            titulo_imagen.append('')
            imagen.append('')  

#extraemos los datos de los articulos de la portada de "EL MUNDO"
for articulo in articulos[3]:
    if articulo.h2 or articulo.h3:
        medio.append(nombres[3])
        time.append(now.isoformat())
        if articulo.h2:
            if 'title' in articulo.h2.a.attrs:
                titulo_articulo.append(articulo.h2.a['title'])
            else:
                titulo_articulo.append(articulo.h2.a.string)
            href_articulo.append(articulo.h2.a['href'])
        else:
            titulo_articulo.append(articulo.h3.a.string)
            href_articulo.append(articulo.h3.a['href'])
        if articulo.find('span',itemprop='name'):
            autor_articulo.append(articulo.find('span',itemprop='name').string)
        else:
            autor_articulo.append('')
        if articulo.img:
            if articulo.img['alt']!='':
                titulo_imagen.append(articulo.img['alt'])
                imagen.append(articulo.img['src'])
                source_url= articulo.img['src']
                folder = folders[3]
                load_requests(source_url, folder)
            elif articulo.figcaption: 
                titulo_imagen.append(articulo.figcaption.get_text())
                imagen.append(articulo.img['src'])
                source_url= articulo.img['src']
                folder = folders[3]
                load_requests(source_url, folder)
            else:
                titulo_imagen.append('')
                imagen.append(articulo.img['src'])
                source_url= articulo.img['src']
                folder = folders[3]
                load_requests(source_url, folder)
        else:
            titulo_imagen.append('')
            imagen.append('')  

In [9]:
#creamos un panda DataFrame
medios = pd.DataFrame({'medio': medio, 
                       'time': time, 
                       'titulo_articulo': titulo_articulo, 
                       'autor_articulo': autor_articulo,
                       'href_articulo': href_articulo, 
                       'titulo_imagen': titulo_imagen, 
                       'imagen': imagen
                      })

In [10]:
#exportamos los datos a un csv
medios.to_csv('medios.csv', encoding='utf-8', index=False)